In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt


Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type

In [32]:
# 我们创建一个以形状电影作为输入的图层
# (n_frames, width, height, channels) and 返回形状相同的电影.
 
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 24, 72, 1),
                   padding='same', return_sequences=True))
#input_shape(None, 24, 72, 40)
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 4, 5),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')




In [33]:
seq.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_45 (ConvLSTM2D) (None, None, 24, 72, 40)  60640     
_________________________________________________________________
batch_normalization_45 (Batc (None, None, 24, 72, 40)  160       
_________________________________________________________________
conv_lst_m2d_46 (ConvLSTM2D) (None, None, 24, 72, 40)  204960    
_________________________________________________________________
batch_normalization_46 (Batc (None, None, 24, 72, 40)  160       
_________________________________________________________________
conv_lst_m2d_47 (ConvLSTM2D) (None, None, 24, 72, 40)  115360    
_________________________________________________________________
batch_normalization_47 (Batc (None, None, 24, 72, 40)  160       
_________________________________________________________________
conv_lst_m2d_48 (ConvLSTM2D) (None, None, 24, 72, 40)

In [3]:
#人工数据生成：
#生成内部具有3到7个移动方块的电影。
#正方形的形状为1x1或2x2像素，
#随时间线性变化。
#为了方便起见，我们首先创建具有更大宽度和高度（80x80）的电影，最后选择40x40窗口。
#For convenience we first create movies with bigger width and height (80x80) and 
#at the end we select a 40x40 window.

def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)

    for i in range(n_samples):
        # 添加3到7个移动方块
        n = np.random.randint(3, 8)

        for j in range(n):
            #初始位置
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # 运动方向
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # Size of the square方块大小
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[i, t, x_shift - w: x_shift + w,
                             y_shift - w: y_shift + w, 0] += 1

                # 通过添加噪声使其更强大。
                # 这个想法是，如果在推理过程中像素的值不完全是一个，
                # 我们需要训练网络的健壮性，并且仍将其视为属于正方形的像素。
                if np.random.randint(0, 2):
                    noise_f = (-1)**np.random.randint(0, 2)
                    noisy_movies[i, t,
                                 x_shift - w - 1: x_shift + w + 1,
                                 y_shift - w - 1: y_shift + w + 1,
                                 0] += noise_f * 0.1

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    # Cut to a 40x40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies



In [4]:
noisy_movies, shifted_movies = generate_movies(n_samples=1200)

In [5]:
print(noisy_movies.shape)

(1200, 15, 40, 40, 1)


In [6]:
print(shifted_movies.shape)
#样本数，15帧，长，宽，channels

(1200, 15, 40, 40, 1)


In [25]:
#data
import numpy as np
import netCDF4
from netCDF4 import Dataset
inp1 = Dataset('CMIP5.input.36mon.1861_2001.nc','r')
'''
target_mon = 1
tg_mn = int(target_mon - 1)
ld_mn1 = int(23 - lead_mon + tg_mn)
ld_mn2 = int(23 - lead_mon + tg_mn + 3)
'''
inpv1 = np.zeros((2,24,24,72))
#四维
#inpv1[:,0:3,:,:] = inp1.variables['sst1'][0:1,0:3,:,:]
#变量
#inpv1[:,3:6,:,:] = inp1.variables['t300'][0:1,0:3,:,:]


inpv1[:,:,:,:] = inp1.variables['sst1'][0:2,0:24,:,:]
#reinp1 = np.swapaxes(inpv1,1,3)  
# (tdim,zdim,ydim,xdim) -> (tdim,xdim,ydim,zdim)
  
trX = inpv1

In [59]:
#data
import numpy as np
import netCDF4
from netCDF4 import Dataset
inp1 = Dataset('CMIP5.input.36mon.1861_2001.nc','r')
inp2 = Dataset('CMIP5.label.12mon.1863_2003.nc','r')

#[字序列数，子序列长度，长，宽]
inpv1 = np.zeros((2960,24,24,72))

#[2960, 24, 24, 72]
#通道压缩到序列中

inpv1[:,0:12,:,:] = inp1.variables['sst1'][0:2960,12:24,:,:]
#c1 [2960, 第二年, :, :, ]
inpv1[:,12:24,:,:] = inp1.variables['t300'][0:2960,12:24,:,:]
#c2 [2960, 第二年, :, :,]


In [60]:
trX = inpv1[:,:,:,:]
trX.shape

(2960, 24, 24, 72)

In [61]:
trX=trX[:, : ,: ,: , np.newaxis]
trX.shape

(2960, 24, 24, 72, 1)

In [58]:
inpv2 = np.zeros((2960,24))
inpv2[:,0:12] = inp2.variables['pr'][0:2960,:,0,0]
inpv2[:,12:24] = inp2.variables['pr'][1:2961,:,0,0]

# [2961, 24]


trY = inpv2[:,:] 
trY.shape

(2960, 24)

In [ ]:
#导入数据
def load_data():
    #CMIP data
    train = Dataset('CMIP5.input.36mon.1861_2001.nc','r')
    label = Dataset('CMIP5.label.12mon.1863_2003.nc','r')
    
    train_sst = train['sst'][:, :12].values# 每年12个月的sst
    train_t300 = train['t300'][:, :12].values
    train_label = label['nino'][:, :].values# label 
    
    #无穷大nan 处理
    train1_t300 = np.nan_to_num(train_t300)
    train1_sst = np.nan_to_num(train_sst)
    
    #SODA data
    train2 = Dataset('SODA.input.36mon.1871_1970','r')
    label2 = Dataset('SODA.label.12mon.1873_1972','r')
    
    train2_sst = train2['sst'][:, :12].values  # (100, 12, 24, 72) 每年12个月的sst
    train2_t300 = train2['t300'][:, :12].values
    train_label2 = label2['nino'][:, 12:36].values# label
    
    print('Train samples: {}, Valid samples: {}'.format(len(train_label), len(train_label2)))
    
    dict_train = {
        'sst':train_sst,
        't300':train_t300,
        
        'label': train_label}
    dict_valid = {
        'sst':train_sst2,
        't300':train_t3002,
        
        'label': train_label2}
    train_dataset = EarthDataSet(dict_train)
    valid_dataset = EarthDataSet(dict_valid)
    return train_dataset, valid_dataset

In [38]:
trY.shape

(2961, 1)

In [9]:
inp2 = Dataset('CMIP5.label.12mon.1863_2003.nc','r')
inpv2 = inp2.variables['pr'][0:1,:,0]
trY = inpv2[:,:]  

In [39]:
print(trX.shape)

(2, 24, 24, 72)


In [40]:
trX=trX[:, : ,: ,: , np.newaxis]

In [41]:
print(trX.shape)

(2, 24, 24, 72, 1)


In [50]:
trX[0:1,0:12]

array([[[[[-0.34927198],
          [-0.42249283],
          [-0.5173344 ],
          ...,
          [-0.44816536],
          [-0.31472501],
          [-0.31864429]],

         [[-0.46517026],
          [-0.51825953],
          [-0.37060675],
          ...,
          [-0.52901304],
          [-0.48909923],
          [-0.37314227]],

         [[-0.42650065],
          [-0.38423744],
          [-0.24157822],
          ...,
          [-0.79590225],
          [-0.70571452],
          [-0.49498636]],

         ...,

         [[ 0.08648874],
          [ 0.19044997],
          [ 0.        ],
          ...,
          [-1.03282261],
          [-0.47016257],
          [-0.0865995 ]],

         [[ 0.14353335],
          [ 0.34460726],
          [ 0.37872037],
          ...,
          [-0.41682637],
          [-0.34333193],
          [ 0.13460237]],

         [[ 0.07848886],
          [ 0.41830871],
          [ 0.31805953],
          ...,
          [-0.40736741],
          [-0.1292223 ],
          

In [65]:
# Train the network
#noisy_movies, shifted_movies = generate_movies(n_samples=1200)

seq.fit(trX[0:2,0:10], trX[0:2,5:15], batch_size=12,
        epochs=50, validation_split=0.05)


Train on 1 samples, validate on 1 samples
Epoch 1/50
1/1 [==============================] - 1s 662ms/step - loss: -0.4119 - val_loss: 0.5881
Epoch 2/50
1/1 [==============================] - 1s 671ms/step - loss: -0.4317 - val_loss: 0.6015
Epoch 3/50
1/1 [==============================] - 1s 653ms/step - loss: -0.5760 - val_loss: 0.5640
Epoch 4/50
1/1 [==============================] - 1s 677ms/step - loss: -0.7397 - val_loss: 0.5701
Epoch 5/50
1/1 [==============================] - 1s 661ms/step - loss: -0.9551 - val_loss: 0.5550
Epoch 6/50
1/1 [==============================] - 1s 775ms/step - loss: -0.8033 - val_loss: 0.5311
Epoch 7/50
1/1 [==============================] - 1s 694ms/step - loss: -1.0031 - val_loss: 0.5132
Epoch 8/50
1/1 [==============================] - 1s 717ms/step - loss: -1.3835 - val_loss: 0.4871
Epoch 9/50
1/1 [==============================] - 1s 703ms/step - loss: -1.5507 - val_loss: 0.4839
Epoch 10/50
1/1 [==============================] - 1s 647ms/step - 

In [66]:
y_pred = seq.predict(trX[0:2,0:10])

In [ ]:
for j in range(16):
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::])
    new = new_pos[::, -1, ::, ::, ::]
    track = np.concatenate((track, new), axis=0)


# And then compare the predictions
# to the ground truth
track2 = noisy_movies[which][::, ::, ::, ::]
for i in range(15):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(121)

    if i >= 7:
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20)

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot)
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = shifted_movies[which][i - 1, ::, ::, 0]

    plt.imshow(toplot)
    plt.savefig('%i_animate.png' % (i + 1))